In [1]:
using Pkg
Pkg.status()
# Pkg.instantiate()

      Status `~/Mathematics/Conferences/Presentations/SumOfSquaresSlides/2022_ICM_Computational_Math/Project.toml`
  [7c1d4256] DynamicPolynomials v0.4.5
  [7073ff75] IJulia v1.23.3
  [4076af6c] JuMP v1.1.1
  [8bc5a954] PermutationGroups v0.3.2
  [c946c3f1] SCS v1.1.2
  [4b9e565b] SumOfSquares v0.6.2
  [24249f21] SymPy v1.1.6
  [858aa9a9] SymbolicWedderburn v0.3.0


# Optimization

In [2]:
using LinearAlgebra
using DynamicPolynomials
using SCS

In [3]:
@polyvar x y
motzkin = x^4*y^2 + y^4*x^2 - 3*x^2*y^2 + 1

x⁴y² + x²y⁴ - 3x²y² + 1

In [4]:
b = monomials([x,y], 0:maxdegree(motzkin)÷2)
A = b*b'
# x^2*y^3 .== A

10×10 Matrix{Monomial{true}}:
 x⁶    x⁵y   x⁴y²  x³y³  x⁵    x⁴y   x³y²  x⁴    x³y   x³
 x⁵y   x⁴y²  x³y³  x²y⁴  x⁴y   x³y²  x²y³  x³y   x²y²  x²y
 x⁴y²  x³y³  x²y⁴  xy⁵   x³y²  x²y³  xy⁴   x²y²  xy³   xy²
 x³y³  x²y⁴  xy⁵   y⁶    x²y³  xy⁴   y⁵    xy³   y⁴    y³
 x⁵    x⁴y   x³y²  x²y³  x⁴    x³y   x²y²  x³    x²y   x²
 x⁴y   x³y²  x²y³  xy⁴   x³y   x²y²  xy³   x²y   xy²   xy
 x³y²  x²y³  xy⁴   y⁵    x²y²  xy³   y⁴    xy²   y³    y²
 x⁴    x³y   x²y²  xy³   x³    x²y   xy²   x²    xy    x
 x³y   x²y²  xy³   y⁴    x²y   xy²   y³    xy    y²    y
 x³    x²y   xy²   y³    x²    xy    y²    x     y     1

In [78]:
# m = Model()
# P = @variable m P[1:length(b), 1:length(b)] Symmetric
# dot(x^3*y^1 .== A, P)

In [6]:
using JuMP

In [7]:
m = Model()
let f = motzkin
    𝕩 = variables(f)
    b = monomials(𝕩, 0:maxdegree(f)÷2)
    lb = length(b)
    A = b*b'
    
    P = @variable m P[1:lb, 1:lb] Symmetric
    @constraint m P in PSDCone()
    
    for t in monomials(𝕩, 0:maxdegree(f))
        A_t = A .== t
        @constraint m dot(A_t, P) == DynamicPolynomials.coefficient(f, t)
    end
    
    m
end
m

A JuMP Model
Feasibility problem with:
Variables: 55
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 28 constraints
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: P

# Using `SumOfSquares.jl`

In [8]:
using SumOfSquares

In [9]:
m = SOSModel()
@constraint m sos_constraint motzkin in SOSCone()
print(m)

In [10]:
set_optimizer(m, SCS.Optimizer)
optimize!(m)
@info termination_status(m)

------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 10, constraints m: 20
cones: 	  z: primal zero / dual free vars: 10
	  s: psd vars: 10, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 20, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.99e+00  2.00e-01  2.40e+00  1.20e+00  1.00e-01  2.52e-04 
    25| 2.80e+13  5.69e+11  3.53e+19  1.76e+19  1.00e-01  4.72e-04 
----------------------------------------

┌ Info: INFEASIBLE
└ @ Main In[10]:3


In [11]:
m = SOSModel()
@constraint m sos_constraint (x^2 + y^2 + 1)*motzkin in SOSCone()
print(m)

In [12]:
set_optimizer(m, SCS.Optimizer)
optimize!(m)
@info termination_status(m)

------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 45, constraints m: 72
cones: 	  z: primal zero / dual free vars: 27
	  s: psd vars: 45, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 90, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.00e+00  2.61e-01  1.87e+00  9.35e-01  1.00e-01  2.23e-04 
    50| 8.10e-07  2.31e-07  1.58e-06  7.88e-07  1.00e-01  2.56e-03 
----------------------------------------

┌ Info: OPTIMAL
└ @ Main In[12]:3


In [13]:
m[:sos_constraint]

sos_constraint : (1)x⁶y² + (2)x⁴y⁴ + (1)x²y⁶ + (-2)x⁴y² + (-2)x²y⁴ + (-3)x²y² + (1)x² + (1)y² + (1) is SOS

In [15]:
sos_dec = sos_decomposition(m[:sos_constraint], 1e-6)
res = polynomial(sos_dec) - (x^2 + y^2 + 1)*motzkin

-8.682134544635289e-8x⁶y² + 6.835648676666016e-16x⁵y³ - 2.3894062328722043e-7x⁴y⁴ - 8.648612836626212e-16x³y⁵ - 8.682134633453131e-8x²y⁶ - 1.7697661164952007e-16x⁵y² - 6.549999014108959e-16x⁴y³ + 1.1089548499072873e-16x³y⁴ - 4.173986419629659e-16x²y⁵ - 4.390374774843053e-8x⁴y² - 3.330669073875474e-16x³y³ - 4.390374686025211e-8x²y⁴ - 4.286850306893944e-16x⁴y + 5.03866362732463e-16x³y² + 4.363376205957347e-16x²y³ - 1.8549507101882013e-16xy⁴ + 3.2694458875364006e-16x³y + 3.04002035989015e-7x²y² + 1.2273676457215796e-15xy³ + 1.3691925531243426e-15x²y + 1.858154127503447e-17xy² + 1.3115902453009198e-7x² - 8.881784197001235e-16xy + 1.3115902386395817e-7y² - 2.7087054202848827e-16x - 3.04435175924639e-16y - 6.583188838504839e-8

In [16]:
norm(res, 1)

1.1325427916466189e-6

In [17]:
@info "" rounded_solution=round(polynomial(sos_dec), digits=6) original_motzkin=(x^2 + y^2 + 1.0)*motzkin

┌ Info: 
│   rounded_solution = x^6*y^2 + 2.0*x^4*y^4 + x^2*y^6 - 2.0*x^4*y^2 - 2.0*x^2*y^4 - 3.0*x^2*y^2 + x^2 + y^2 + 1.0
│   original_motzkin = x^6*y^2 + 2.0*x^4*y^4 + x^2*y^6 - 2.0*x^4*y^2 - 2.0*x^2*y^4 - 3.0*x^2*y^2 + x^2 + y^2 + 1.0
└ @ Main In[17]:1


In [23]:
G = gram_matrix(m[:sos_constraint]);
# @show norm(G.Q - round.(G.Q, digits = 6), 1)
round.(G.Q, digits=4)

9×9 Matrix{Float64}:
  1.0   0.0   0.5  -0.0   0.0  -1.5   0.0   0.0  -0.0
  0.0   1.0  -0.0  -0.0   0.0  -0.0  -0.0   0.0  -1.0
  0.5  -0.0   1.0  -0.0   0.0  -1.5   0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   1.0   0.0   0.0   0.0  -1.0   0.0
  0.0   0.0   0.0   0.0   1.0  -0.0  -1.0   0.0  -0.0
 -1.5  -0.0  -1.5   0.0  -0.0   3.0  -0.0   0.0   0.0
  0.0  -0.0   0.0   0.0  -1.0  -0.0   1.0  -0.0  -0.0
  0.0   0.0  -0.0  -1.0   0.0   0.0  -0.0   1.0   0.0
 -0.0  -1.0  -0.0   0.0  -0.0   0.0  -0.0   0.0   1.0

In [27]:
g = let m = G.basis.monomials
    m' * round.(Int, 2 .* G.Q) * m
end
(g, )

(2x⁶y² + 4x⁴y⁴ + 2x²y⁶ - 4x⁴y² - 4x²y⁴ - 6x²y² + 2x² + 2y² + 2,)

In [29]:
g == 2(x^2 + y^2 + 1)*motzkin

true

## Finding the denominator

In [31]:
m = SOSModel()
# X = DynamicPolynomials.monomials([x,y], 2:2)
X = [x^2, y^2]

2-element Vector{Monomial{true}}:
 x²
 y²

In [32]:
@variable m denom Poly(X);
print(denom)

(_[1])*x^2 + (_[2])*y^2

In [33]:
@constraint m sum(DynamicPolynomials.coefficients(denom)) >= 1

_[1] + _[2] ≥ 1.0

In [34]:
@variable m t
@objective m Max t

t

In [35]:
@constraint m (denom-t)*motzkin ∈ SOSCone()

(_[1])x⁶y² + (_[1] + _[2])x⁴y⁴ + (_[2])x²y⁶ + (-3 _[1] - t)x⁴y² + (-3 _[2] - t)x²y⁴ + (3 t)x²y² + (_[1])x² + (_[2])y² + (-t) is SOS

In [37]:
m

A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`Vector{AffExpr}`-in-`SumOfSquares.SOSPolynomialSet{FullSpace, Monomial{true}, MonomialVector{true}, SumOfSquares.Certificate.Newton{SOSCone, MonomialBasis, Tuple{}}}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: denom, t

In [38]:
set_optimizer(m, SCS.Optimizer)
optimize!(m)
@info termination_status(m)

------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 48, constraints m: 73
cones: 	  z: primal zero / dual free vars: 27
	  l: linear vars: 1
	  s: psd vars: 45, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 104, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 7.85e+00  1.00e+00  8.27e+00 -3.59e+00  1.00e-01  1.10e-04 
    50| 2.74e-07  6.10e-09  2.74e-07 -1.37e-07  1.00e-01  1.31e-03 
------------------

┌ Info: OPTIMAL
└ @ Main In[38]:3


In [39]:
denominator = round.(value(m[:denom]), digits=5)

1.33422x² + 1.33422y²

In [40]:
objective_value(m)

2.7385546976901366e-7